**Brief Introduction**

This project aims to build a predictive model to forecast Financial Distress at Small and Medium Enterprises in Vietnam based on different financial ratios where data is available. The data were preprocessed for missing values and outliers. Logistic Regression is used for each year of the dataset to obtain the results for each one separately.

### I. Data preparation

The data used for this project comprises of 5 datasets from year 2008 to year 2012, extracted from the "Vietnam Enterprises Inspection" conducted by the General Statistics Office of Vietnam. Each datasets have been filtered to only include data of Small and Medium Enterprises. 

The code below presents the steps I took to clean each dataset and concatenate them into a panel dataset, which takes 'ma_thue' as the ID variable, representing each company, and 'year' as the time variable. Since the dataset of each year differs in terms of code assignment to featured attributes, it is necessary to read the datasets and extract the relevant columns in each year separately.

In [65]:
# Import libraries
import pandas as pd
import pyreadstat

In [66]:
# Year 2008
data08, meta = pyreadstat.read_dta('/Users/kieuanhle/Documents/6. Research/Dữ liệu/0. Data/SME/New/sme2008.dta', encoding = 'latin1')
df08 = data08[['ma_thue','ts22','ts32','ts42','ts12','ts11','kqkd4','kqkd9','ts222','ts232','kqkd13']]
df08.loc[:, ['netincome']] = df08.loc[:, ['kqkd9']] - df08.loc[:, ['kqkd13']]
df08 = df08.drop(columns='kqkd13')
df08.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205689 entries, 0 to 205688
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   ma_thue    205689 non-null  object 
 1   ts22       204555 non-null  object 
 2   ts32       171010 non-null  object 
 3   ts42       178860 non-null  object 
 4   ts12       205417 non-null  object 
 5   ts11       197465 non-null  object 
 6   kqkd4      205122 non-null  object 
 7   kqkd9      202441 non-null  object 
 8   ts222      190211 non-null  object 
 9   ts232      205285 non-null  object 
 10  netincome  0 non-null       float64
dtypes: float64(1), object(10)
memory usage: 17.3+ MB


In [67]:
# Year 2009
data09, meta = pyreadstat.read_dta('/Users/kieuanhle/Documents/6. Research/Dữ liệu/0. Data/SME/New/sme2009.dta', encoding = 'latin1')
df09 = data09[['ma_thue','ts22','ts32','ts42','ts12','ts11','kqkd4','kqkd17','ts222','ts232','kqkd19']]
df09.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248555 entries, 0 to 248554
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   ma_thue  248555 non-null  object 
 1   ts22     247793 non-null  object 
 2   ts32     217571 non-null  object 
 3   ts42     220946 non-null  object 
 4   ts12     248417 non-null  object 
 5   ts11     239339 non-null  object 
 6   kqkd4    247931 non-null  object 
 7   kqkd17   246725 non-null  float64
 8   ts222    235555 non-null  object 
 9   ts232    248260 non-null  object 
 10  kqkd19   246423 non-null  float64
dtypes: float64(2), object(9)
memory usage: 20.9+ MB


In [68]:
# Year 2010
data10, meta = pyreadstat.read_dta('/Users/kieuanhle/Documents/6. Research/Dữ liệu/0. Data/SME/New/sme2010.dta', encoding = 'latin1')
df10 = data10[['ma_thue','ts22','ts32','ts42','ts12','ts11','kqkd4','kqkd17','ts222','ts232','kqkd19']]

In [69]:
# Year 2011
data11, meta = pyreadstat.read_dta('/Users/kieuanhle/Documents/6. Research/Dữ liệu/0. Data/SME/New/sme2011.dta', encoding = 'latin1')
df11 = data11[['ma_thue','ts22','ts32','ts42','ts12','ts11','kqkd4','kqkd20','ts392','ts402','kqkd22']]

In [70]:
# Year 2012
data12, meta = pyreadstat.read_dta('/Users/kieuanhle/Documents/6. Research/Dữ liệu/0. Data/SME/New/sme2012.dta', encoding = 'latin1')
df12 = data12[['ma_thue','ts22','ts32','ts42','ts12','ts11','kqkd5','kqkd22','ts202','ts212','kqkd25']]

In [71]:
# Rename columns
# Define a list of new column names based on their index
new_column_names = ['ma_thue', 'currentassets', 'accountreceivables', 'inventory', 'totalassets_ck', 'totalassets_dk','netsales','EBITDA','totaldebt','totalequity','netincome']
df08.columns = new_column_names
df09.columns = new_column_names
df10.columns = new_column_names
df11.columns = new_column_names
df12.columns = new_column_names

In [72]:
# Add years to the datasets
df08.loc[:, ['year']] = 2008
df09.loc[:, ['year']] = 2009
df10.loc[:, ['year']] = 2010
df11.loc[:, ['year']] = 2011
df12.loc[:, ['year']] = 2012

In [73]:
# Concatenate all dataframes to make a panel dataset
panel = pd.concat([df08,df09, df10, df11, df12], axis=0, ignore_index=True)
# Sort the dataset
panel.sort_values(by=['ma_thue', 'year'], inplace=True)
# Reset the index
panel.reset_index(drop=True, inplace=True)

In [74]:
# View the merged dataset
panel.info()
panel.describe()
panel.describe(include=['O'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1442557 entries, 0 to 1442556
Data columns (total 12 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   ma_thue             1442557 non-null  object 
 1   currentassets       1434730 non-null  object 
 2   accountreceivables  1242524 non-null  object 
 3   inventory           1270040 non-null  object 
 4   totalassets_ck      1438683 non-null  object 
 5   totalassets_dk      1384758 non-null  object 
 6   netsales            1429573 non-null  object 
 7   EBITDA              1423257 non-null  object 
 8   totaldebt           1348030 non-null  object 
 9   totalequity         1437461 non-null  object 
 10  netincome           1220208 non-null  float64
 11  year                1442557 non-null  int64  
dtypes: float64(1), int64(1), object(10)
memory usage: 132.1+ MB


,ma_thue,currentassets,accountreceivables,inventory,totalassets_ck,totalassets_dk,netsales,EBITDA,totaldebt,totalequity
count,1442557,1434730.0,1242524,1270040,1438683.0,1384758,1429573,1423257,1348030,1437461
unique,775279,114791.0,71729,68420,136403.0,126132,130877,66261,108967,99139
top,Ch­a cã,5305.2,0,0,6208.2,0,0,0,0,1000
freq,32383,2759.0,109619,97316,2759.0,40436,90614,220243,53315,4917


In [75]:
# Delete mismatching data records and duplicates
panel['ma_thue'] = pd.to_numeric(panel['ma_thue'], errors='coerce')
panel.dropna(subset=['ma_thue'], inplace=True)
# Drop rows that have both the same ma_thue and year
panel.drop_duplicates(subset=['ma_thue','year'],inplace=True)

In [76]:
# Add variable cash, which is the rest of current assets after subtracting account receivables and inventory
panel['cash'] = panel['currentassets'] - panel['accountreceivables'] - panel['inventory']

In [77]:
# Delete missing values
panel.dropna()
panel.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1401680 entries, 1189 to 1410163
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   ma_thue             1401680 non-null  float64
 1   currentassets       1396255 non-null  object 
 2   accountreceivables  1213681 non-null  object 
 3   inventory           1251204 non-null  object 
 4   totalassets_ck      1398545 non-null  object 
 5   totalassets_dk      1345846 non-null  object 
 6   netsales            1389482 non-null  object 
 7   EBITDA              1386970 non-null  object 
 8   totaldebt           1319174 non-null  object 
 9   totalequity         1397460 non-null  object 
 10  netincome           1192135 non-null  float64
 11  year                1401680 non-null  int64  
 12  cash                1150397 non-null  object 
dtypes: float64(2), int64(1), object(10)
memory usage: 149.7+ MB


In [78]:
# Remove data records where totalassets is recorded at 0
panel = panel[panel['totalassets_ck']>0]
panel = panel.astype(float)
panel.describe()

,ma_thue,currentassets,accountreceivables,inventory,totalassets_ck,totalassets_dk,netsales,EBITDA,totaldebt,totalequity,netincome,year,cash
count,1.396972e+06,1.394782e+06,1.212237e+06,1.249779e+06,1.396972e+06,1.343316e+06,1.387541e+06,1.385092e+06,1.317739e+06,1.396045e+06,1.190327e+06,1.396972e+06,1.148976e+06
mean,4.756414e+08,2.270491e+04,7.871425e+03,5.338694e+03,3.793296e+04,3.154800e+04,2.718792e+04,9.847670e+02,2.432726e+04,1.499400e+04,7.146785e+02,2.010287e+03,9.307116e+03
std,1.052113e+09,8.407992e+05,2.690085e+05,7.369452e+04,1.375894e+06,1.186708e+06,6.050608e+05,1.232925e+05,1.072570e+06,6.403121e+05,1.082327e+05,1.376553e+00,4.999126e+05
min,0.000000e+00,-1.070610e+05,-1.362020e+07,-1.267310e+05,1.000000e-01,-3.993100e+04,-3.774760e+05,-3.907295e+07,-1.402537e+07,-2.840929e+07,-3.907295e+07,2.008000e+03,-8.461900e+04
25%,3.044434e+07,9.810000e+02,1.380000e+02,1.610000e+02,1.415000e+03,1.130000e+03,3.150000e+02,-4.700000e+00,3.740000e+02,7.660000e+02,-5.700000e+00,2.009000e+03,3.860000e+02
50%,1.052558e+08,2.545000e+03,6.880000e+02,7.120000e+02,3.514000e+03,2.904000e+03,1.631000e+03,3.100000e+00,1.643000e+03,1.635000e+03,5.000000e-01,2.010000e+03,9.385000e+02
75%,3.600352e+08,6.741975e+03,2.328000e+03,2.196000e+03,9.227000e+03,7.692000e+03,6.848000e+03,5.600000e+01,5.312000e+03,3.854000e+03,4.000000e+01,2.012000e+03,2.288000e+03
max,8.077246e+09,3.293083e+08,1.266053e+08,2.332164e+07,5.235253e+08,4.694168e+08,2.706071e+08,4.873520e+07,4.898996e+08,2.856620e+08,4.821433e+07,2.012000e+03,2.304475e+08


The financial ratios used for forecasting include the followings:
- **Cash holdings ratio**: A higher cash holdings ratio generally indicates that a company has a strong liquidity position, which can provide a cushion during economic downturns or unexpected expenses. However, excessively high cash holdings might also signify inefficiency in utilizing resources for investment or growth.
- **Return on Assets (ROA)**: ROA indicates how efficiently a company utilizes its assets to generate profits. A higher ROA signifies better asset utilization and management, indicating that the company is more effective in converting its assets into profits.
- **Return on Equity (ROE)**: ROE reflects a company's ability to generate profits from the shareholders' investments. A higher ROE indicates that the company is generating more profit per unit of shareholders' equity, making it more attractive to potential investors.
- **Asset Turnover**: Asset turnover shows how many dollars of revenue a company generates for each dollar of assets it holds. A higher asset turnover suggests that the company is using its assets efficiently to generate sales.
- **Debt Ratio**: The debt ratio helps assess a company's leverage and financial risk. A lower debt ratio indicates that the company relies less on debt financing and is considered less risky, while a higher debt ratio implies greater reliance on debt, which may increase financial vulnerability in challenging economic conditions.

In [79]:
# Add ratio variables
panel['cashholdings'] = panel['cash']/panel['totalassets_ck'] # Cash holdings ratio
panel['ROA'] = panel['EBITDA']/panel['totalassets_ck'] # Profitability ratio - ROA
panel['ROE'] = panel['EBITDA']/panel['totalequity'] # Profitability ratio - ROE
panel['assetturnover'] = panel['netsales']/((panel['totalassets_ck'] - panel['totalassets_dk'])/2) # Efficiency ratio - Asset turnover
panel['debtratio'] = panel['totaldebt']/panel['totalassets_ck'] # Debt ratio

In [80]:
# Remove outliers
def remove_outliers_iqr(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    return data[~((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR)))]

panel = panel.apply(remove_outliers_iqr)

In [81]:
panel.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1396972 entries, 1189 to 1410163
Data columns (total 18 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   ma_thue             1260693 non-null  float64
 1   currentassets       1227421 non-null  float64
 2   accountreceivables  1060502 non-null  float64
 3   inventory           1102851 non-null  float64
 4   totalassets_ck      1222675 non-null  float64
 5   totalassets_dk      1174780 non-null  float64
 6   netsales            1196100 non-null  float64
 7   EBITDA              1016901 non-null  float64
 8   totaldebt           1150697 non-null  float64
 9   totalequity         1221470 non-null  float64
 10  netincome           845868 non-null   float64
 11  year                1396972 non-null  float64
 12  cash                1015999 non-null  float64
 13  cashholdings        1080401 non-null  float64
 14  ROA                 1015658 non-null  float64
 15  ROE              

Financial distress are identified based on criteria:
- Negative equity
- Negative net income
- Debt ratio being greater than 1 for 2 consecutive years

In [82]:
## Add distress variable
# Check if debtratio in two consecutive years are biggert than 1
panel['insolvency'] = 1
for i in range(1, len(panel)):
    if panel['debtratio'].iloc[i] > 1 and panel['debtratio'].iloc[i - 1] > 1:
        panel['insolvency'].iloc[i] = 0

# Define a custom function to set the value of dummy variable financial distress
def assign_value(row):
    if row['netincome'] < 0 or row['debtratio'] > 1:
        return 1
    elif row['netincome'] < 0 and row['insolvency'] == 1: 
        return 1
    elif row['debtratio'] > 1 and row['insolvency'] == 1:
        return 1
    else:
        return 0
panel['distress'] = panel.apply(assign_value, axis=1)

/var/folders/yr/4qbw35_x54zfg0_tgg4h2r300000gn/T/ipykernel_3163/3541097980.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel['insolvency'].iloc[i] = 0
/var/folders/yr/4qbw35_x54zfg0_tgg4h2r300000gn/T/ipykernel_3163/3541097980.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  panel['insolvency'].iloc[i] = 0
/var/folders/yr/4qbw35_x54zfg0_tgg4h2r300000gn/T/ipykernel_3163/3541097980.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

In [83]:
panel.info()
panel.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1396972 entries, 1189 to 1410163
Data columns (total 20 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   ma_thue             1260693 non-null  float64
 1   currentassets       1227421 non-null  float64
 2   accountreceivables  1060502 non-null  float64
 3   inventory           1102851 non-null  float64
 4   totalassets_ck      1222675 non-null  float64
 5   totalassets_dk      1174780 non-null  float64
 6   netsales            1196100 non-null  float64
 7   EBITDA              1016901 non-null  float64
 8   totaldebt           1150697 non-null  float64
 9   totalequity         1221470 non-null  float64
 10  netincome           845868 non-null   float64
 11  year                1396972 non-null  float64
 12  cash                1015999 non-null  float64
 13  cashholdings        1080401 non-null  float64
 14  ROA                 1015658 non-null  float64
 15  ROE              

,ma_thue,currentassets,accountreceivables,inventory,totalassets_ck,totalassets_dk,netsales,EBITDA,totaldebt,totalequity,netincome,year,cash,cashholdings,ROA,ROE,assetturnover,debtratio,insolvency,distress
1189,2006317.0,7936.0,2903.0,2863.0,8449.0,8091.0,3176.0,23.0,3728.0,4721.0,NaN,2008.0,2170.0,0.256835,0.002722,0.004872,17.743017,0.441236,1,0
1190,0.0,496.0,0.0,0.0,531.0,1043.0,1846.0,NaN,27.0,504.0,NaN,2008.0,496.0,NaN,NaN,NaN,-7.210938,0.050847,1,0
1191,0.0,NaN,1994.0,1192.0,16424.0,3061.0,2714.0,111.0,5015.0,NaN,83.0,2009.0,NaN,0.804554,0.006758,0.009729,0.406196,0.305346,1,0
1194,0.0,1609.0,191.0,NaN,3865.0,3423.0,1726.0,NaN,743.0,3122.0,NaN,2010.0,NaN,NaN,NaN,NaN,7.809955,0.192238,1,0
1197,0.0,823.0,467.5,49.5,1545.5,1408.0,37.5,-0.1,248.5,1297.0,-0.1,2012.0,306.0,0.197994,-0.000065,-0.000077,0.545455,0.160789,1,1


In [84]:
panel['distress'].value_counts()

distress
0    1182509
1     214463
Name: count, dtype: int64

### II. Forecasting

Logistic regression is a popular statistical technique for forecasting problems where the dependent variable is binary, specifically, the financial distress status in this study. The output of the model is the result of financial distress, corresponding to the input features X. 

In [85]:
# Import necessary libraries
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [86]:
## Define a subroutine to train model
def forecast(year):
    # Load data for the specified year
    data = pd.read_csv('panel.csv')
    data = data[data['year'] == year]

    # Remove rows with infinite or NaN values
    data = data[np.isfinite(data)]
    data.dropna(inplace=True)

    # Set 'ma_thue' and 'year' columns as the index
    data = data.set_index(['ma_thue', 'year'])

    # Separate features (X) and target variable (y)
    X = data.drop(columns=['distress'])
    y = data['distress']
    y = y.astype('category')

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train the Model
    model = LogisticRegression(max_iter=300000)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the Model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

    print(f"Results for year {year}:")
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("ROC-AUC Score:", roc_auc)


In [87]:
## Run for results
# Year 2008
forecast(2008)

Results for year 2008:
Accuracy: 0.928847392144237
Precision: 0.9931434599156118
Recall: 0.7954377442179744
F1 Score: 0.8833636310326629
ROC-AUC Score: 0.9823595284505826


*Interpretation*
- The model achieved an accuracy of approximately 92.88% for the year 2008, indicating that it made correct predictions for a large portion of the companies.
- The precision of 99.31% suggests that when the model predicted a company as distressed, it was highly accurate.
- The recall score of 79.54% implies that the model correctly identified about 79.54% of the actual distressed companies.
- The F1 score of 88.34% combines precision and recall, showing a good balance between the two metrics.
- The ROC-AUC score of 98.24% indicates that the model has a high ability to distinguish between distressed and non-distressed companies.

In [88]:
# Year 2009
forecast(2009)

Results for year 2009:
Accuracy: 0.8080377765231227
Precision: 0.9974306269270298
Recall: 0.3419661733615222
F1 Score: 0.5093151403831016
ROC-AUC Score: 0.9694982451629947


*Interpretation*
- The accuracy of approximately 80.80% suggests that the model's performance for the year 2009 is lower compared to other years.
- The precision of 99.74% indicates that the model has a very high ability to correctly identify distressed companies when it predicts them.
- The recall score of 34.20% suggests that the model has a relatively low ability to identify actual distressed companies.
- The F1 score of 50.93% is relatively low, indicating that there is an imbalance between precision and recall.
- The ROC-AUC score of 96.95% still shows that the model can differentiate between distressed and non-distressed companies effectively.

In [89]:
# Year 2010
forecast(2010)

Results for year 2010:
Accuracy: 0.8725998424086592
Precision: 0.9969414893617021
Recall: 0.5505213687766192
F1 Score: 0.7093386318478571
ROC-AUC Score: 0.8950749920202294


*Interpretation*
- The accuracy of approximately 87.26% suggests that the model performs better in 2010 compared to 2009 but slightly lower than 2008 and 2011.
- The precision of 99.69% indicates that the model has a high ability to correctly identify distressed companies when it makes predictions.
- The recall score of 55.05% suggests that the model identifies about 55.05% of actual distressed companies.
- The F1 score of 70.93% is an improvement over the previous year, indicating a better balance between precision and recall.
- The ROC-AUC score of 89.51% suggests a moderate ability to differentiate between distressed and non-distressed companies.

In [90]:
# Year 2011
forecast(2011)

Results for year 2011:
Accuracy: 0.9716623915964334
Precision: 0.9940077128448531
Recall: 0.9405490372200078
F1 Score: 0.9665397484712126
ROC-AUC Score: 0.9830943014928423


*Interpretation*
- The accuracy of approximately 97.17% indicates that the model performs very well in 2011, making correct predictions for most companies.
- The precision of 99.40% suggests that the model has a high ability to correctly identify distressed companies when it makes predictions.
- The recall score of 94.05% is very high, indicating that the model effectively identifies most of the actual distressed companies.
- The F1 score of 96.65% is high, indicating a well-balanced performance between precision and recall.
- The ROC-AUC score of 98.31% suggests a high ability to differentiate between distressed and non-distressed companies.

In [91]:
# Year 2012
forecast(2012)

Results for year 2012:
Accuracy: 0.8875116207003408
Precision: 0.9988149318585818
Recall: 0.42325075326414463
F1 Score: 0.5945564634648168
ROC-AUC Score: 0.9588884762157336


*Interpretation*
- The accuracy of approximately 88.75% suggests that the model performs well in 2012 but is slightly lower than some previous years.
- The precision of 99.88% indicates that the model has a very high ability to correctly identify distressed companies when it makes predictions.
- The recall score of 42.33% suggests that the model has a relatively low ability to identify actual distressed companies.
- The F1 score of 59.46% indicates a moderate balance between precision and recall.
- The ROC-AUC score of 95.89% suggests a high ability to differentiate between distressed and non-distressed companies.

**Overall Evaluation**

In general, the model's performance varies across different years, with some years showing very high accuracy, precision, recall, and F1 scores, while others show slightly lower performance. The ROC-AUC scores indicate that the model is generally effective at distinguishing between distressed and non-distressed companies.

In reality, it's essential to consider the specific context and business requirements when evaluating the model's performance for different years. Moreover, further analysis and fine-tuning may be necessary to improve the model's performance in specific years if needed.